<a href="https://www.kaggle.com/code/javihm77/sentiment-analysis-por-favor-no-se-enoje?scriptVersionId=234565709" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **SENTIMENT ANALYSIS POR FAVOR NO SE ENOJE**

This notebook presents a sentiment analysis of YouTube comments for the Guatemalan radio program "Por Favor No Se Enoje." The program focuses on political discussions in Guatemala and neighboring countries. This project is part of [5-Day Gen AI Intensive Course with Google](https://rsvp.withgoogle.com/events/google-generative-ai-intensive_2025q1/home).

![](https://img.rephonic.com/artwork/por-favor-no-se-enoje.jpg?width=600&height=600&quality=95)

In [1]:
import numpy as np
import pandas as pd
import os
from google import genai
from google.genai import types
from IPython.display import HTML, Markdown, display
import plotly.express as px
import plotly.graph_objects as go
import time


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/por-favor-no-se-enoje-guatemala-youtube-comments/raw_youtube_comments.csv


# Dataset and EDA

**Obtaining YouTube Comments via API**

To understand the process of collecting YouTube comments using the YouTube Data API, please refer to the accompanying [Jupyter Notebook](https://colab.research.google.com/drive/1tuQrYcucH1AToTm-7FXH7t3IZgTXdYoP?usp=sharing). This notebook provides a step-by-step guide and the Python code used for data extraction.

**Dataset on Kaggle**

The resulting dataset of YouTube comments is publicly available on Kaggle. You can access and explore the data through the following link: [POR FAVOR NO SE ENOJE - Guatemala YouTube Comments](https://www.kaggle.com/datasets/javihm77/por-favor-no-se-enoje-guatemala-youtube-comments). This dataset contains the comments extracted from YouTube videos related to the "Por Favor No Se Enoje" program in Guatemala.

In [2]:
df = pd.read_csv("/kaggle/input/por-favor-no-se-enoje-guatemala-youtube-comments/raw_youtube_comments.csv")
df.head()

,Unnamed: 0,author,updated_at,like_count,text,video_id,public,video title,video published_date,video view_count,video like_count,video dislike_count,video comment_count
0,621,@carolnavarro5510,2024-05-30 14:28:22-06:00,3,Muy buena idea😅😅 el negocio de los stickers 😂,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-29 18:00:00-06:00,3595,181,0,6
1,620,@delilahcurtis4956,2024-05-30 15:13:57-06:00,1,Tanto Dineroooo!!! Por eso la mayoría del cír...,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-29 18:00:00-06:00,3595,181,0,6
2,619,@aurapalencia2273,2024-05-30 17:54:28-06:00,1,Excelente exposición del tema,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-29 18:00:00-06:00,3595,181,0,6
3,618,@OscarLopezSandoval-mp2wp,2024-05-30 18:12:57-06:00,0,Quique... donde estan los delfines ?,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-29 18:00:00-06:00,3595,181,0,6
4,617,@MaynorGarcia-lf8rj,2024-05-31 06:53:58-06:00,1,Grand exponente. Lo mejor de Guate.,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-29 18:00:00-06:00,3595,181,0,6


In [3]:
# Calculate the value counts of video titles
videos = df["video title"].value_counts().nlargest(15)

# Create a Pandas DataFrame for Plotly Express, including the publish date
videos_df = pd.DataFrame({'Video Title': videos.index, 'Number of Comments': videos.values})

# Merge with the original DataFrame to get the publish date for the top videos
videos_df = videos_df.merge(df[['video title', 'video published_date']].drop_duplicates(subset=['video title']),
                           left_on='Video Title', right_on='video title', how='left')
videos_df = videos_df.drop(columns=['video title'])

# Create the bar plot using Plotly Express with custom hover data and labels
fig = px.bar(videos_df,
             x="Video Title",
             y="Number of Comments",
             title="Top 15 Videos with Most Comments",
             labels={"Video Title": "Video Title", "Number of Comments": "Number of Comments",
                     "video published_date": "Published Date"}, # Label for the hover data
             hover_data={"Video Title": True, "Number of Comments": True, "video published_date": True})

# Rotate x-axis labels for better readability
fig.update_layout(xaxis_tickangle=-90)

# Show the plot
fig.show()

In [4]:
# Calculate the value counts of videos published on each date and sort by date
views = df["video published_date"].value_counts().sort_index()

# Convert the Series to a DataFrame for Plotly Express
views_df = pd.DataFrame({'Published Date': views.index, 'Number of Comments': views.values})

# Create the line plot using Plotly Express
fig = px.line(views_df,
              x="Published Date",
              y="Number of Comments",
              title='Number of Comments Published by Date',
              labels={'Published Date': 'Published Date', 'Number of Comments': 'Number of Comments'})

# Rotate x-axis labels for better readability
fig.update_layout(xaxis_tickangle=-90)

# Remove top and right spines (similar to the Seaborn/Matplotlib style)
fig.update_layout(
    xaxis_showline=True,
    yaxis_showline=True,
    xaxis_linecolor='black',
    yaxis_linecolor='black',
    showlegend=False,
    plot_bgcolor='white'
)
fig.update_xaxes(gridcolor='lightgray')
fig.update_yaxes(gridcolor='lightgray')

# Show the plot
fig.show()

In [5]:
# Create the histogram using Plotly Express
fig = px.histogram(df,
                   x="like_count",
                   nbins=8,
                   title='Distribution of Like Counts',
                   labels={'like_count': 'Like Count'},
                   marginal="rug") # Optional: Add a rug plot for individual data points

# Remove top and right spines (similar to Matplotlib) and set background
fig.update_layout(
    xaxis_showline=True,
    yaxis_showline=True,
    xaxis_linecolor='black',
    yaxis_linecolor='black',
    plot_bgcolor='white'
)
fig.update_xaxes(gridcolor='lightgray')
fig.update_yaxes(gridcolor='lightgray')

# Show the plot
fig.show()

# 1.Sentiment Analysis of Comments using Gemini (Vertex AI)

In this section, we perform sentiment analysis on the extracted YouTube comments to understand the overall sentiment expressed towards the "Por Favor No Se Enoje" program. We leverage the power of the Gemini model, accessed through Vertex AI, for this task.

The process involves the following steps:

1.  **Setting up the Gemini API Client:** We first install the necessary `google-genai` library and authenticate using an API key securely retrieved from Kaggle Secrets. We also configure a retry mechanism to handle potential API errors.
2.  **Defining the Sentiment Analysis Function (`sentiment`):** This function takes a comment as input and constructs a zero-shot prompt for the Gemini model. The prompt instructs the model to classify the comment's sentiment as either "POSITIVO", "NEGATIVO", or "NEUTRAL", and to return only the classification.
3.  **Making Predictions:** The `generate_content` method of the Gemini model is called with the constructed prompt to obtain the sentiment prediction for each comment. Error handling is included to manage potential issues during API calls.
4.  **Applying Sentiment Analysis to the Dataset:** The `sentiment` function is then applied to the "text" column of our DataFrame, creating a new "sentiment" column containing the predicted sentiment for each comment.

For example, the comment "Grand exponente. Lo mejor de Guate. Muy bonito e ilustrativo programa" is classified as: POSITIVO

In [6]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

## Model configuration

In [7]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("5GenAIDays")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [8]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=4,
)

def sentiment(comment):
    zero_shot_prompt = """Clasifica el siguiente comentario de YouTube sobre un programa de política como POSITIVO, NEGATIVO o NEUTRAL. Devuelve ÚNICAMENTE la clasificación, sin texto adicional.

Comentario: "{}"

Sentimiento (POSITIVO, NEGATIVO o NEUTRAL):""".format(comment)

    #print(zero_shot_prompt)
    time.sleep(0.01)
    
    try:
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            config=model_config,
            contents=zero_shot_prompt
        )
        return response.text.strip()
        
    except Exception as e:
        print(f"Error al procesar el comentario: '{comment}'. Error: {e}")
        return None

comm = "Grand exponente. Lo mejor de Guate. Muy bonito e ilustrativo programa"
res = sentiment(comm)
print(res)
    

POSITIVO


## Classification

In [9]:
from tqdm import tqdm

tqdm.pandas(desc="Analyzing Sentiment")  # Initialize tqdm for pandas
df["sentiment"] = df["text"].progress_map(sentiment)

Analyzing Sentiment: 100%|██████████| 1246/1246 [06:57<00:00,  2.99it/s]


In [10]:
pd.set_option('display.max_colwidth', None)

df[["text","sentiment"]].head()

,text,sentiment
0,Muy buena idea😅😅 el negocio de los stickers 😂,NEUTRAL
1,Tanto Dineroooo!!! Por eso la mayoría del círculo del gobierno de turno se enriquecen porque NO hay quién controle los dis que gastos que hacen,NEGATIVO
2,Excelente exposición del tema,POSITIVO
3,Quique... donde estan los delfines ?,NEUTRAL
4,Grand exponente. Lo mejor de Guate.,POSITIVO


In [11]:
# Count the occurrences of each sentiment
sentiment_counts = df['sentiment'].value_counts()

# Create the pie chart
fig = go.Figure(data=[go.Pie(labels=sentiment_counts.index,
                             values=sentiment_counts.values,
                             hoverinfo='label+percent',
                             textinfo='value',
                             textfont_size=20,
                             marker=dict(line=dict(color='#000000', width=2)))])

# Update the layout for better presentation (optional)
fig.update_layout(title='Sentiment Distribution')
fig.update_traces(hole=.3)  # Add a donut hole (optional)

# Show the chart
fig.show()

# 2.Storing Data in SQLite and Natural Language Querying with Gemini

This section outlines the process of saving the processed YouTube comment data into a local SQLite database and then enabling natural language querying of this data using the Gemini model through Vertex AI's function calling capabilities.

## Creating and Populating the SQLite Database

In [12]:
%load_ext sql
%sql sqlite:///pfnse.db

In [13]:
%%sql

DROP TABLE IF EXISTS comments;

CREATE TABLE IF NOT EXISTS comments (
    comment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    author VARCHAR(255),
    text TEXT,
    video_title VARCHAR(255),
    video_published_date DATETIME,
    like_count INTEGER,
    video_view_count INTEGER,
    video_like_count INTEGER,
    video_dislike_count INTEGER,
    updated_at DATETIME,
    sentiment VARCHAR(255)
);

 * sqlite:///pfnse.db
Done.
Done.


[]

In [14]:
import sqlite3

db_file = "pfnse.db"
db_conn = sqlite3.connect(db_file)

## Insert data from Dataframe

In [15]:
try:
    cursor = db_conn.cursor()
    for index, row in df.iterrows():
        cursor.execute("INSERT INTO comments (author, text, video_title, video_published_date, like_count, video_view_count,video_like_count,video_dislike_count, updated_at, sentiment) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                        (row['author'], row['text'], row['video title'], row['video published_date'], row['like_count'], row['video view_count'], row['video like_count'], row['video dislike_count'], row['updated_at'], row['sentiment']))
    db_conn.commit()  # Commit the changes within the 'with' block
except sqlite3.Error as e:
    print(f"Database error: {e}")


## Defining Database Interaction Functions

In [16]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("comments")

 - DB CALL: describe_table(comments)


[('comment_id', 'INTEGER'),
 ('author', 'VARCHAR(255)'),
 ('text', 'TEXT'),
 ('video_title', 'VARCHAR(255)'),
 ('video_published_date', 'DATETIME'),
 ('like_count', 'INTEGER'),
 ('video_view_count', 'INTEGER'),
 ('video_like_count', 'INTEGER'),
 ('video_dislike_count', 'INTEGER'),
 ('updated_at', 'DATETIME'),
 ('sentiment', 'VARCHAR(255)')]

In [17]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables()


['sqlite_sequence', 'comments']

In [18]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from comments limit 2")

 - DB CALL: execute_query(select * from comments limit 2)


[(1,
  '@carolnavarro5510',
  'Muy buena idea😅😅 el negocio de los stickers 😂',
  'LA OTRA CORTE CELESTIAL - PFNSE',
  '2024-05-29 18:00:00-06:00',
  3,
  3595,
  181,
  0,
  '2024-05-30 14:28:22-06:00',
  'NEUTRAL'),
 (2,
  '@delilahcurtis4956',
  'Tanto Dineroooo!!! Por eso la mayoría  del círculo del gobierno de turno  se enriquecen porque NO hay quién controle los dis que gastos que hacen',
  'LA OTRA CORTE CELESTIAL - PFNSE',
  '2024-05-29 18:00:00-06:00',
  1,
  3595,
  181,
  0,
  '2024-05-30 15:13:57-06:00',
  'NEGATIVO')]

## Gemini for Natural Language Querying

In [19]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
for YouTube program comments. To answer the user's questions, you will use the following steps:
1. Use the 'list_tables' tool to see what tables are available.
2. Use the 'describe_table' tool to understand the schema of the relevant table(s), including column names and types.
3. Use the 'execute_query' tool to issue an SQL SELECT query to retrieve the necessary data.
4. Once you have the data, you will answer the user's question.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query."""

client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

In [20]:
resp = chat.send_message("What is the percentage distribution and quantity of comments by sentiment?")
print(f"\n{resp.text}")

 - DB CALL: execute_query(SELECT sentiment, COUNT(*) AS count, CAST(COUNT(*) AS REAL) * 100 / (SELECT COUNT(*) FROM Comments) AS percentage FROM Comments GROUP BY sentiment)

The comments are distributed as follows: 64.37% Negative (802 comments), 16.85% Neutral (210 comments), and 18.78% Positive (234 comments).


In [21]:
resp = chat.send_message("What is the top 5 of video titles with most views? Indicate how many views each video has")
print(f"\n{resp.text}")


I am sorry, I cannot fulfill this request. The available tools do not allow me to access video titles or view counts. I can only access and query the 'Comments' table.



In [22]:
resp = chat.send_message("What is the top 10 of authors with more comments? indicate how many comments the person did")
print(f"\n{resp.text}")

 - DB CALL: execute_query(SELECT author, COUNT(*) AS count FROM Comments GROUP BY author ORDER BY count DESC LIMIT 10)

Here are the top 10 authors with the most comments and the number of comments they made:
1. @CARLOSSTREMS51: 40 comments
2. @erapsa: 37 comments
3. @BetoCuevasGT: 24 comments
4. @tulisof: 18 comments
5. @ottodomingo8956: 18 comments
6. @juanfranciscobarillasbarriento: 18 comments
7. @anaaguirre8985: 18 comments
8. @jorgeaguilar-bq3uj: 17 comments
9. @DavidVasquez-cn2zm: 17 comments
10. @beatrizalvarez1567: 15 comments



In [29]:
resp = chat.send_message("What is the top 10 of text comments with more like counts? indicate how many likes they had")
print(f"\n{resp.text}")

 - DB CALL: execute_query(SELECT text, like_count FROM Comments ORDER BY like_count DESC LIMIT 10)

Here are the top 10 text comments with the most likes and their like counts:

1.  "Pintan al gobierno como que tiene que estar suplicando a la oposición. No me gustó este programa" - 18 likes
2.  "Guatemala necesita que CICIG vuelva y reinicie la lucha anti corrupción, el pacto de corruptos encabezado por la fundación contra el terrorismo ya tiene cooptadas todas las instituciones de justicia, especialmente el MP. El que nada debe nada teme. 👍⚖️" - 17 likes
3.  "En cuanto más ataquen a semilla, estos orates no comprenden que le hace propaganda a Semilla" - 15 likes
4.  "¡Exijo la inmediata libertad al periodista Sr. José Rubén Zamora!" - 15 likes
5.  "Disculpen yo insisto, que no quieren que , se gobierne tranquilamente" - 14 likes
6.  "No la gente no quiere ver eso. Eso es lo que USTEDES quieren ver. Les gusta poner en problemas al presidente Arevalo...hasta por un p💨" - 12 likes
7.  "E

# 3.Google Search

**Web Search for Program Info**

This section retrieves information about the YouTube political program "Por Favor No Se Enoje" and its founders using Google Search via the Gemini model.

We configure Gemini with the `GoogleSearch` tool and define `query_with_grounding` to ask for a summary and founder descriptions. The model performs a web search and generates a response.

The output displays:

1.  A Markdown summary of the program and its founders.
2.  HTML content from the Google Search results used for grounding.

This provides external context about the program discussed in the comments.

In [25]:
from google.genai import types

# And now re-run the same query with search grounding enabled.
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

def query_with_grounding():
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents="Provide a summary in a paragrah of the political program in Guatemala: Por Favor No se Enoje and give us a description of the founders",
        config=config_with_search,
    )
    return response.candidates[0]


rc = query_with_grounding()
Markdown(rc.content.parts[0].text)

"Por Favor No Se Enoje" (Please Don't Get Angry) is a Guatemalan political program that provides information, analysis, context, opinions, satire, and studies of current political events. It is available as a podcast and on YouTube. It aims to inform the public about the political landscape in Guatemala.

I am unable to find information about the founders.


In [26]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)